In [930]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [931]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [932]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(training_set)

# obtengo los nodos del grafo, esta vez los nodos son los usuarios y no los problemas:
nodes = training_set.user_id.unique()

# creo un diccionario que va a tener a los usuarios como keys y los valores seran los
# problemas que ha hecho ese usuario
grouped = training_set.groupby('user_id')['problem_id'].apply(list)

# muestra el numero de usuarios
print(len(nodes))

# muestra la lista de problemas que ha hecho cada usuario
print(grouped)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [933]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [934]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [935]:
# In[3]:

def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de usuarios
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de problemas que ha hecho cada usuario
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado


# In[4]:

def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    


# In[5]:

def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo


In [936]:
# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

16805


In [937]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    

In [938]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped.index, 'list_problem_id':grouped.values})

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)

      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [939]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))

      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [940]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users_recommend = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users_recommend)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [941]:

# en df_new tengo los usuarios a los que tengo que hacer recomendaciones

# primero guardo la lista de usuarios
user_list_recomend = df_users_recommend.user_id.unique()

# creo el nuevo dataframe con los resultados 
column_user_recomend = {'user_id': user_list_to_recommend}
dataframe_user_recomend = pd.DataFrame.from_dict(column_user_recomend)

print(dataframe_user_recomend)

    user_id
0        18
1        25
2        60
3       130
4       414
5       443
6       448
7       689
8       912
9       935
10     1711
11     1893
12     1952
13     1955
14     2025
15     2027
16     2038
17     2041
18     2051
19     2096
20     2106
21     2120
22     2257
23     2355
24     2373
25     2576
26     2626
27     2906
28     2966
29     3118
..      ...
35     3708
36     3710
37     3712
38     3714
39     3722
40     3724
41     3727
42     3731
43     3735
44     3749
45     3759
46     3779
47     3784
48     3795
49     3842
50     3846
51     3847
52     3855
53     3859
54     3873
55     3880
56     3890
57     3931
58     3937
59     3938
60     3941
61     3946
62     3961
63     3963
64     3969

[65 rows x 1 columns]


In [942]:
def apply_wcn(row, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    cn_list = nx.common_neighbors(graph, row['one'], row['two'])
    
    value_wcn = sum([graph[row['one']][x]['weight'] + graph[row['two']][x]['weight'] for x in cn_list])
    
    return value_wcn

def create_wcn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de wcn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_wcn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_wcn['wcn'] = dataFrame_wcn.apply (lambda row: apply_wcn(row, graph), axis=1)


    return dataFrame_wcn


In [943]:
wcn_df = create_wcn_data(graph, nodes)
print(wcn_df)


          one   two  wcn
0           5     6    0
1           5     9    0
2           5     8    0
3           5    16    0
4           5    17    0
5           5    12    0
6           5    15    0
7           5    22    0
8           5     7    0
9           5    28    0
10          5    29    0
11          5    32    0
12          5    35    0
13          5    33    0
14          5    31    0
15          5    40    0
16          5    49    0
17          5    42    0
18          5    46    0
19          5    24    0
20          5    53    0
21          5    60    0
22          5    59    0
23          5    66    0
24          5    68    0
25          5    71    0
26          5    69    0
27          5    70    0
28          5    65    0
29          5    73    0
...       ...   ...  ...
2012991  3699  2046    0
2012992  3699  4018    0
2012993  3997  4011    0
2012994  3997  3951    0
2012995  3997  3949    0
2012996  3997  3952    0
2012997  3997  3982    0
2012998  3997  2046    0


In [944]:
def getCommonNeighbors(row, wcn_df):
    """
        Funcion que devuelve la lista de los usuarios de ese usuario que tienen vecinos en comun
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = wcn_df[wcn_df['one'] == row['user_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['wcn'] > 0]
    column_result_two_tmp = wcn_df[wcn_df['two'] == row['user_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['wcn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_wcn = list_one + list_two
    
    # print(list_wcn)
    
    if list_wcn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_wcn = graph.nodes()
        list_wcn.remove(row['user_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_wcn


In [945]:
dataframe_user_recomend['neighbors'] = dataframe_user_recomend.apply (lambda row: getCommonNeighbors(row, wcn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_user_recomend)

    user_id                                          neighbors
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...
14     2025  [2120, 2053, 2041, 2086, 2095, 2168, 2169,

In [946]:
def wcn_value(one, two, graph):
    """
        Funcion que devuelve el valor wcn en comun de esos dos nodos
    """
    cn_list = nx.common_neighbors(graph, one, two)
    
    value_wcn = sum([graph[one][x]['weight'] + graph[two][x]['weight'] for x in cn_list])
    
    return value_wcn

In [947]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de wcn de todos los nodos con los que tiene wcn
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    user = row['user_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + wcn_value(user, elem, graph)
        
    return suma

In [948]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada usuario vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con user_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    user = row['user_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = wcn_value(user, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [949]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['total_weight'] = dataframe_user_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_user_recomend)


    user_id                                          neighbors  total_weight
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...         19540
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...        311354
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...        291024
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...        955183
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...        770258
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...        726274
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...        318218
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...        506634
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...        120444
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...       1109946
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...       1248693
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...        158495

In [950]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['score'] = dataframe_user_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

# de esta forma en dataframe_user_recomend voy a tener las ponderaciones para cada vecino 
print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...         19540   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...        311354   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...        291024   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...        955183   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...        770258   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...        726274   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...        318218   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...        506634   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...        120444   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...       1109946   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...       1248693   
11     1893  [1910, 1919, 1574, 1911, 19

In [951]:
def getProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
   
    # lista resultante de la concatenacion de las listas de problemas de los usuarios similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_problems
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [952]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
dataframe_user_recomend['list_problems'] = dataframe_user_recomend.apply (lambda row: getProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...         19540   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...        311354   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...        291024   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...        955183   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...        770258   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...        726274   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...        318218   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...        506634   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...        120444   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...       1109946   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...       1248693   
11     1893  [1910, 1919, 1574, 1911, 19

In [953]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_problems'] if not (x in conjunto_vacio or function_add(x))]

In [954]:
# ahora tengo la lista de posibles problemas a recomendar para cada usuario
# tengo que contar el numero de apariciones de cada problema en la lista de problemas

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de problemas sin repeticiones
dataframe_user_recomend['lista_problemas_unique'] = dataframe_user_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...         19540   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...        311354   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...        291024   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...        955183   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...        770258   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...        726274   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...        318218   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...        506634   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...        120444   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...       1109946   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...       1248693   
11     1893  [1910, 1919, 1574, 1911, 19

In [955]:
# elimino la columna de la lista de problemas (la que tiene repeticiones), ya que no interesa
del dataframe_user_recomend['list_problems']

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...         19540   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...        311354   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...        291024   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...        955183   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...        770258   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...        726274   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...        318218   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...        506634   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...        120444   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...       1109946   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...       1248693   
11     1893  [1910, 1919, 1574, 1911, 19

In [956]:
# elimino la columna de total_weight, ya que no interesa
del dataframe_user_recomend['total_weight']

print(dataframe_user_recomend)

    user_id                                          neighbors  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...   
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...   
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...   
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...   
14     202

In [957]:
def getListProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [958]:
# ahora, para la lista de problemas que se pueden recomendar, tengo que hacer la suma de las ponderaciones
# voy a crear una columna en la cual se guarde una lista de listas de problemas, en las que cada posicion coincidira con el 
# usuario que las haya hecho, sin que se guarden los problemas que ha hecho el usuario al que quiero recomendar
dataframe_user_recomend['lista_problemas_por_user'] = dataframe_user_recomend.apply(lambda row: getListProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(dataframe_user_recomend)


    user_id                                          neighbors  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...   
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...   
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...   
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...   
14     202

In [959]:

# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
df_separation = dataframe_user_recomend.groupby(['user_id']).lista_problemas_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['user_id', 'recommendation']

print(df_separation)

      user_id  recommendation
0          18              62
1          18             502
2          18             509
3          18             105
4          18             307
5          18             166
6          18              39
7          18               2
8          18               6
9          18               8
10         18              10
11         18              13
12         18              33
13         18             139
14         18              35
15         18              44
16         18              49
17         18             150
18         18             183
19         18             327
20         18             191
21         18             325
22         18             390
23         18             254
24         18             134
25         18             162
26         18             109
27         18             253
28         18              78
29         18             171
...       ...             ...
9923     3969             505
9924     3

In [960]:
def getScoring(row, df_users_recommend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de problemas que han hecho los vecinos del usuario
    list_problems_users_df = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_per_user = list(list_problems_users_df['lista_problemas_por_user'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_user = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el problema del que quiero calcular el scoring
    problem = row['recommendation']
    # print(problem)
    
    suma = 0
    k = len(list_problems_per_user)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_problems_per_user[i]
        # print(lista_a_comprobar)
        if problem in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + list_scores_per_user[i]
    
    return suma

In [961]:
# ahora voy a calcular el scoring para cada problema
df_separation['score'] = df_separation.apply(lambda row: getScoring(row, dataframe_user_recomend), axis=1)

print(df_separation)

      user_id  recommendation     score
0          18              62  0.246520
1          18             502  0.144831
2          18             509  0.123132
3          18             105  0.179836
4          18             307  0.217861
5          18             166  0.071341
6          18              39  0.544063
7          18               2  0.384186
8          18               6  0.209007
9          18               8  0.111105
10         18              10  0.157369
11         18              13  0.261157
12         18              33  0.387666
13         18             139  0.223951
14         18              35  0.166274
15         18              44  0.445394
16         18              49  0.350051
17         18             150  0.354094
18         18             183  0.395599
19         18             327  0.375486
20         18             191  0.258291
21         18             325  0.327482
22         18             390  0.279887
23         18             254  0.370010


In [962]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['user_id', 'score'], ascending=False)
print(df_separation)

      user_id  recommendation     score
9842     3969              44  0.501248
9801     3969             109  0.462555
9841     3969              33  0.437890
9877     3969             327  0.420749
9791     3969               2  0.396433
9884     3969             162  0.393652
9873     3969             254  0.388480
9878     3969             159  0.386999
9845     3969              49  0.384345
9870     3969             187  0.374278
9829     3969             258  0.357455
9874     3969             141  0.345716
9869     3969             136  0.344235
9817     3969             209  0.330770
9816     3969             203  0.320655
9883     3969             155  0.303419
9795     3969              13  0.287681
9902     3969             390  0.284425
9846     3969              70  0.275459
9827     3969             256  0.272830
9796     3969             253  0.268552
9815     3969             195  0.262984
9798     3969              62  0.262572
9882     3969             139  0.257883


In [963]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

print(df_recommend)

                                            recommendation  \
user_id                                                      
18       [39, 44, 109, 183, 33, 2, 327, 254, 150, 49, 2...   
25       [39, 44, 109, 183, 33, 2, 254, 327, 150, 49, 1...   
60       [39, 44, 109, 183, 33, 49, 162, 258, 187, 325,...   
130      [254, 49, 258, 253, 256, 259, 307, 251, 510, 3...   
414      [39, 159, 390, 253, 256, 70, 441, 307, 471, 25...   
443      [33, 49, 162, 134, 187, 136, 203, 155, 256, 13...   
448      [39, 44, 109, 183, 2, 327, 254, 150, 49, 258, ...   
689      [150, 327, 49, 254, 258, 187, 325, 203, 256, 3...   
912      [39, 109, 44, 183, 33, 254, 327, 49, 258, 150,...   
935      [39, 44, 183, 150, 13, 237, 251, 336, 4, 29, 1...   
1711     [259, 251, 309, 437, 308, 53, 352, 442, 252, 3...   
1893     [44, 109, 183, 33, 254, 327, 150, 162, 258, 18...   
1952     [39, 150, 109, 254, 49, 2, 159, 187, 134, 258,...   
1955     [39, 44, 183, 33, 2, 254, 150, 49, 258, 159, 1...   
2025    

In [964]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation'][:k])
        return value
    else:
        return row['recommendation'][:k]

In [965]:
k = 10
# ahora saco los k mejores problemas para cada usuario
df_recommend['k_recommendation'] = df_recommend.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend)

                                            recommendation  \
user_id                                                      
18       [39, 44, 109, 183, 33, 2, 327, 254, 150, 49, 2...   
25       [39, 44, 109, 183, 33, 2, 254, 327, 150, 49, 1...   
60       [39, 44, 109, 183, 33, 49, 162, 258, 187, 325,...   
130      [254, 49, 258, 253, 256, 259, 307, 251, 510, 3...   
414      [39, 159, 390, 253, 256, 70, 441, 307, 471, 25...   
443      [33, 49, 162, 134, 187, 136, 203, 155, 256, 13...   
448      [39, 44, 109, 183, 2, 327, 254, 150, 49, 258, ...   
689      [150, 327, 49, 254, 258, 187, 325, 203, 256, 3...   
912      [39, 109, 44, 183, 33, 254, 327, 49, 258, 150,...   
935      [39, 44, 183, 150, 13, 237, 251, 336, 4, 29, 1...   
1711     [259, 251, 309, 437, 308, 53, 352, 442, 252, 3...   
1893     [44, 109, 183, 33, 254, 327, 150, 162, 258, 18...   
1952     [39, 150, 109, 254, 49, 2, 159, 187, 134, 258,...   
1955     [39, 44, 183, 33, 2, 254, 150, 49, 258, 159, 1...   
2025    

In [966]:
# elimino las columnas que no me interesan
del df_recommend['recommendation']
del df_recommend['score']

print(df_recommend)

                                          k_recommendation
user_id                                                   
18            [39, 44, 109, 183, 33, 2, 327, 254, 150, 49]
25            [39, 44, 109, 183, 33, 2, 254, 327, 150, 49]
60          [39, 44, 109, 183, 33, 49, 162, 258, 187, 325]
130      [254, 49, 258, 253, 256, 259, 307, 251, 510, 316]
414       [39, 159, 390, 253, 256, 70, 441, 307, 471, 251]
443        [33, 49, 162, 134, 187, 136, 203, 155, 256, 13]
448          [39, 44, 109, 183, 2, 327, 254, 150, 49, 258]
689      [150, 327, 49, 254, 258, 187, 325, 203, 256, 390]
912         [39, 109, 44, 183, 33, 254, 327, 49, 258, 150]
935           [39, 44, 183, 150, 13, 237, 251, 336, 4, 29]
1711     [259, 251, 309, 437, 308, 53, 352, 442, 252, 314]
1893      [44, 109, 183, 33, 254, 327, 150, 162, 258, 187]
1952        [39, 150, 109, 254, 49, 2, 159, 187, 134, 258]
1955          [39, 44, 183, 33, 2, 254, 150, 49, 258, 159]
2025        [39, 109, 33, 2, 187, 159, 134, 258, 136, 20

In [967]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [968]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()
list_recom_problems = df_recommend['k_recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [969]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [970]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [971]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [972]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [973]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [974]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [975]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.8615384615384616
Precision ----------
0.2476923076923076
Mrr  ----------
0.49661782661782655
Recall  ----------
0.2526098910567792
F1  ----------
0.23641871783788593


In [976]:

f = open("C:/hlocal/TFM/vot_pond", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()